# Select your IPTS 

In [ ]:
from __code import system
system.System.select_working_dir()
from __code.__all import custom_style
custom_style.style()

# Description 

The user will select a profile region (rectangle). The pixels in the smallest width of the rectangle will be added. Then the profile ploted for each of the image index. 

__Example of application:__
This will allow for example to follow the flow of water as it enters a crack.

# Python Import 

# Select Images

In [ ]:
# import ipywe.fileselector
# from NeuNorm.normalization import Normalization
from __code.profile_vs_file_index import ProfileVsFileIndex

%matplotlib notebook

In [ ]:
o_profile = ProfileVsFileIndex(working_dir = system.System.get_working_dir())
o_profile.select_images()

In [ ]:
# working_dir = system.System.get_working_dir()
# folder_ui = ipywe.fileselector.FileSelectorPanel(instruction='Select Input Images ...',
#                                                               start_dir=working_dir,
#                                                               multiple=True)

# folder_ui.show()        

In [ ]:
# list_data_files = folder_ui.selected

In [ ]:
# # load data
# o_load = Normalization()
# o_load.load(file=list_data_files, notebook=True)

In [ ]:
o_profile.load_images()

# Select Profile Region 

In [ ]:
o_profile.select_profile()

In [ ]:
# def calculate_integrated_sample(data):
#     if len(data)>1:
#         integrated_array = np.array([_array for _array in data])
#         return integrated_array.mean(axis=0)
#     else:
#         return np.squeeze(data)

In [ ]:
# integrated_data = calculate_integrated_sample(o_load.data['sample']['data'])

In [ ]:
# from ipywidgets.widgets import interact
# from ipywidgets import widgets

# import matplotlib.pyplot as plt
# import matplotlib.patches as patches


In [ ]:
# # import numpy as np
# [height, width] = np.shape(integrated_data)

# x_left = 0
# y_top = 0


In [ ]:
# def plot_images_with_box(x_left, y_top, width, height):
    
#     data = integrated_data
    
#     fig_box = plt.figure(figsize=(5, 5))
#     ax_img = plt.subplot(111)
    
#     ax_img.imshow(data, cmap = 'rainbow',
#                   interpolation = None)
# #                   vmin = min_intensity,
# #                   vmax = max_intensity)
#     ax_img.add_patch(patches.Rectangle((x_left, y_top), width, height, fill=False))
    
#     return [x_left, y_top, width, height]

# profile = interact(plot_images_with_box,
#                   x_left = widgets.IntSlider(min = 0, 
#                                              max = image_width-1, 
#                                              step= 1, 
#                                              value = x_left,
#                                              continuous_update = False),
#                   y_top = widgets.IntSlider(min = 0, 
#                                             max = image_height-1, 
#                                             step = 1, 
#                                             value = y_top,
#                                             continuous_update = False), 
#                   width = widgets.IntSlider(min = 0, 
#                                             max = image_width-1, 
#                                             step = 1, 
#                                             value = width,
#                                             continuous_update = False), 
#                   height = widgets.IntSlider(min = 0, 
#                                              max = image_height-1, 
#                                              step = 1, 
#                                              value = height,
#                                              continuous_update = False))
    

# Calculate Integrated Profile vs Images

In [ ]:
[roi_left, roi_top, roi_width, roi_height] = profile.widget.result

In [ ]:
sample_data = o_load.data['sample']['data']

w = widgets.IntProgress()
w.max = len(sample_data)-1
display(w)
index = 0

profile_array = []
for _image in sample_data:
    _profile_image = _image[roi_top:roi_top+roi_height, roi_left:roi_left+roi_width]
    _value = np.sum(_profile_image)
    profile_array.append(_value)
    w.value = index
    index += 1

# Calculate Profile vs Images 

In [ ]:
rebin = 1

w = widgets.IntProgress()
w.max = len(sample_data)-1
display(w)
index = 0
rebin_range = np.arange(0, roi_height-roi_top, rebin)

profile_1d = []
def calculate_1d_profiles():
    
    for _index, _array in enumerate(sample_data):
        _roi_array = _array[roi_top:roi_top + roi_height, roi_left:roi_left + roi_width]
        _width_profile = np.sum(_roi_array,1)
        rebin_width_profile = [sum(_width_profile[x:x+rebin]) for x in rebin_range]
        profile_1d.append(rebin_width_profile)
        _index += 1
        w.value = _index
    
calculate_1d_profiles()

In [ ]:
def plot_profile(file_index):
    
    data_1d = profile_1d[file_index]
    data_2d = sample_data[file_index]
    
    fig = plt.figure(figsize=(5,5))
    
    ax_plt = plt.subplot(211)
    ax_plt.plot(data_1d)
    ax_plt.set_title(os.path.basename(list_files[file_index]))
    
    ax_img = plt.subplot(212)
    ax_img.imshow(data_2d, cmap = 'rainbow',
                  interpolation = None)
    ax_img.add_patch(patches.Rectangle((roi_left, roi_top), roi_width, roi_height, fill=False))
    

number_of_files = len(sample_data)
range_selection = interact(plot_profile,
                          file_index = widgets.IntSlider(min=0, 
                                                         max=number_of_files-1, 
                                                         value=0, 
                                                         step=1, 
                                                         description="Image Index",
                                                         continuous_update=False))

# Load file_name vs time_stamp file 

In [ ]:
file_ui = ipywe.fileselector.FileSelectorPanel(instruction='Select file_name_vs_time_stamp File ...',
                                                              start_dir=working_dir,
                                                              multiple=False)

file_ui.show()  

In [ ]:
import pandas as pd

In [ ]:
file_vs_timestamp_file = file_ui.selected
if file_vs_timestamp_file:
    if type(file_vs_timestamp_file) == list:
        file_vs_timestamp_file = file_vs_timestamp_file[0]

    df = pd.read_csv(file_vs_timestamp_file)
    
df.head()

In [ ]:
list_file_name = df['#filename'].values
list_column = df.columns.values
list_time_stamp = df[list_column[1]].values

# Output Profiles 

In [ ]:
output_folder_ui = ipywe.fileselector.FileSelectorPanel(instruction='Select Output Folder ...',
                                                       start_dir = working_dir,
                                                       type='directory')
output_folder_ui.show()

In [ ]:
import shutil

# make name of output folder 
output_folder = os.path.join(os.path.abspath(output_folder_ui.selected), os.path.basename(os.path.dirname(list_file_name[0]) + '_profiles'))
if os.path.isdir(output_folder):
    shutil.rmtree(output_folder)
os.mkdir(output_folder)


In [ ]:
file0 = list_data_files[0]
df.index[df['#filename'] == file0].tolist()[0]

In [ ]:
def get_time_stamp(file_name):
    index = df.index[df['#filename'] == file_name].tolist()[0]
    return list_time_stamp[index]

In [ ]:
import datetime

In [ ]:
def output_ascii_file(filename = None, data = None):
    f = open(filename, 'w')
    for _line in data:
        f.write(_line)
    f.close()

In [ ]:
time_0 = get_time_stamp(list_data_files[0])

for _index, _profile in enumerate(profile_1d):
    _file_name = list_data_files[_index]
    _time_stamp = get_time_stamp(_file_name)

    ascii_contain = []
    ascii_contain.append("#Image: {}\n".format(_file_name))
    ascii_contain.append("#ROI selected (y0, x0, height, width): ({}, {}, {}, {})\n".format(roi_top, roi_left, roi_height, roi_width))
    ascii_contain.append("#Rebin in y direction: {}\n".format(rebin))
    
    _time_stamp_str = datetime.datetime.fromtimestamp(_time_stamp).strftime("%Y-%m-%d %H:%M:%S")
    ascii_contain.append("#Time Stamp: {}\n".format(_time_stamp_str))
    
    _delta_time = _time_stamp - time_0
    ascii_contain.append("#Delta Time (s): {}\n".format(_delta_time))
    ascii_contain.append("#\n")
    
    ascii_contain.append("#Label: pixel_index, counts\n")
    
    for _index, value in enumerate(profile_1d[_index]):
        ascii_contain.append("{}, {}\n".format(rebin_range[_index], value))
    
    _base_file_name = os.path.basename(_file_name)
    [base, pref] = os.path.splitext(_base_file_name)
    
    output_file_name = os.path.join(output_folder, base + '.txt')
    output_ascii_file(filename=output_file_name, data=ascii_contain)
    
    